In [4]:
import pandas as pd
from experiments.models.GPTModel import GPTInference

In [ ]:
'''
First check which sentence the highlighted text matches 
Use sentence and evidence as pairs that match 
Liberal -- if evidence in NLI shortlisted then rate 1, also use f-1 
'''

In [6]:
def get_annotations():
    annotations_path = '/home/sanjana/factual_evaluation_source_based/annotations/round1_aggredialfact_annotated/dialoguefact_two_annotators_round1.csv'
    df = pd.read_csv(annotations_path)
    return df


In [7]:
df = get_annotations()
df.head()

,Unnamed: 0,uuid,user_id,docid,model,nonfactual_spans,evidence,error_type,summary,dialogue
0,0,b6273d21-9a0c-4033-8e2b-908d0a98c143,sanjana,13611370,gpt4-32k-0613,NaN,NaN,NaN,Derek asks Judy if she can feed his animals on...,"Derek: Judy, r you leaving for the weekend?\r\..."
1,1,ec577599-01d2-4ed5-af77-245326c3ac8b,ann_ccry,13611370,gpt4-32k-0613,lans to,Maybe,Intrinsic_Error,Derek asks Judy if she can feed his animals on...,"Derek: Judy, r you leaving for the weekend?\r\..."
2,2,22a9198f-5a54-4d53-a630-552083d5b350,ann_ccry,13611426,gpt4-32k-0613,both,principally the night,Intrinsic_Error,"Poppy and Harry both prefer Lisbon, especially...",Poppy: Yeah. I definitely prefer Lisbon\r\nHar...
3,3,ef458bfe-4bef-430c-8d97-8efc30b21ded,ann_ccry,13611442,gpt4-32k-0613,NaN,NaN,NaN,"Helen was trying to reach Simon, but he was un...","Helen: Hey, Simo, are you there?\r\nSimon: Yep..."
4,4,e861c609-240f-416e-8a33-c0188daee831,ann_ccry,13611442,gpt4-32k-0613,NaN,NaN,NaN,"Helen was trying to reach Simon, but he was un...","Helen: Hey, Simo, are you there?\r\nSimon: Yep..."


In [8]:
df_errors = df[(~df['error_type'].isnull()) & (~df['evidence'].isnull())]
df_errors.head()[:1]

,Unnamed: 0,uuid,user_id,docid,model,nonfactual_spans,evidence,error_type,summary,dialogue
1,1,ec577599-01d2-4ed5-af77-245326c3ac8b,ann_ccry,13611370,gpt4-32k-0613,lans to,Maybe,Intrinsic_Error,Derek asks Judy if she can feed his animals on...,"Derek: Judy, r you leaving for the weekend?\r\..."


In [28]:
last_row_idx = 1
import spacy
import nltk
import re
import string
nlp = spacy.load('en_core_web_sm')

def get_atomic_facts_gpt(gpt_model_atomic, model, text, text_type):
    instr = f'Convert the {text_type} into facts without adding any unsupported details.'
    prompt = f'{instr}\nDialogue: {text}\nFacts:'
    print(prompt)
    gpt_response = gpt_model_atomic.get_chatgpt_response(prompt, model = model)
    # print(nltk.sent_tokenize(gpt_response))
    return gpt_response
    
class GPTCall():
    def __init__(self):
        self.gpt_model_atomic = GPTInference()

    def get_atomic_facts(self, dlg, gpt_model_type = 'gpt-4-32k-0613'):
        atomic_facts = get_atomic_facts_gpt(self.gpt_model_atomic, gpt_model_type, dlg, 'dialogue')
        # print(atomic_facts)
        # atomic_facts = nltk.sent_tokenize(atomic_facts)
        atomic_facts = list(nlp(atomic_facts).sents)
        atomic_facts = [each.text for each in atomic_facts]
        atomic_facts = [re.sub('[0-9]', '', each) for each in atomic_facts]
        atomic_facts = [each.strip(string.punctuation).strip() for each in atomic_facts]
        atomic_facts = [each for each in atomic_facts if each]
        return atomic_facts 
        

    
def get_summary_sents(doc):
    sents = list(nlp(doc).sents)
    sents = [each.text for each in sents]
    return sents

gpt_call = GPTCall()
def get_dialogue_sents(dlg):
    dlg_sents = dlg.split('\n')
    atomic_facts = gpt_call.get_atomic_facts(dlg)
    
    return dlg_sents, atomic_facts
    

def get_membership_sents(span, src_sent):
    membership_sents = []
    for sent in sents:
        if sent in src:
            membership_sents.append(
            
nonfactual_sentences = []
for summ_sent in summary_sents:
    if nonfactual_spans in summ_sent:
        nonfactual_sentences.append(summ_sent)

evidence_uttx = []
for dlg_sent in dlg_sents:
    if evidence in dlg_sent:
        evidence_uttx.append(dlg_sent)
y_evidence_uttx = []
y_summary_sentences = []
for idx, row in df_errors[:last_row_idx].iterrows():
    nonfactual_spans = row['nonfactual_spans']
    evidence = row['evidence']
    summary_sents = get_summary_sents(row['summary'])
    dlg_sents, atomic_facts = get_dialogue_sents(row['dialogue'])
    


Convert the dialogue into facts without adding any unsupported details.
Dialogue: Derek: Judy, r you leaving for the weekend?
Judy: Nah
Derek: So can you feed my animals on Friday and Saturday?
Judy: sure, no problem
Derek: Thank you :) Maybe on Thursday I would give you my keys?
Judy: Ok
Derek: Thanks :) Later I will tell you the details :)
Judy: surely
Facts:


('lans to',
 'Maybe',
 ["Derek asks Judy if she can feed his animals on Friday and Saturday as she isn''t leaving for the weekend.",
  'She agrees and Derek plans to give her his keys on Thursday.',
  'He also promises to provide her with more details later.'])

In [67]:
instruction = 'Extract only those turns from the snippet that are relevant to the sentence'
prompt = f'{instruction}\nDialogue: {dlg}\nSentence: {summary_sents[1]}\nRelevant Turns:'
print(prompt)
response = GPTInference().get_chatgpt_response(prompt)
print(response)

Extract only those turns from the snippet that are relevant to the sentence
Dialogue: Derek: Judy, r you leaving for the weekend?
Judy: Nah
Derek: So can you feed my animals on Friday and Saturday?
Judy: sure, no problem
Derek: Thank you :) Maybe on Thursday I would give you my keys?
Judy: Ok
Derek: Thanks :) Later I will tell you the details :)
Judy: surely
Sentence: She agrees and Derek plans to give her his keys on Thursday.
Relevant Turns:
Derek: So can you feed my animals on Friday and Saturday?
Judy: sure, no problem
Derek: Thank you :) Maybe on Thursday I would give you my keys?
Judy: Ok
Derek: Thanks :) Later I will tell you the details :)
Judy: surely


In [66]:
instruction = 'Below is a sentence along with a dialogue. List all turns within the dialogue that partially or entirely support the sentence'

prompt = f'{instruction}\nDialogue: {dlg}\nSentence: {summary_sents[1]}\nSupportive Turns:'
print(prompt)
response = GPTInference().get_chatgpt_response(prompt)
print(response)

Below is a sentence along with a dialogue. List all turns within the dialogue that partially or entirely support the sentence
Dialogue: Derek: Judy, r you leaving for the weekend?
Judy: Nah
Derek: So can you feed my animals on Friday and Saturday?
Judy: sure, no problem
Derek: Thank you :) Maybe on Thursday I would give you my keys?
Judy: Ok
Derek: Thanks :) Later I will tell you the details :)
Judy: surely
Sentence: She agrees and Derek plans to give her his keys on Thursday.
Supportive Turns:
Judy: sure, no problem
Derek: Thank you :) Maybe on Thursday I would give you my keys?
Judy: Ok
Derek: Thanks :) Later I will tell you the details :)
Judy: surely


In [58]:
response

'Derek: Judy, r you leaving for the weekend?\nJudy: Nah\nDerek: So can you feed my animals on Friday and Saturday?\nJudy: sure, no problem'

In [63]:
# evidence_uttx_true = 
nonfactual_sentences = []
for summ_sent in summary_sents:
    if nonfactual_spans in summ_sent:
        nonfactual_sentences.append(summ_sent)

evidence_uttx = []
for dlg_sent in dlg_sents:
    if evidence in dlg_sent:
        evidence_uttx.append(dlg_sent)
evidence_uttx

['Derek: Thank you :) Maybe on Thursday I would give you my keys?\r']

In [64]:
nonfactual_sentences

['She agrees and Derek plans to give her his keys on Thursday.']

In [65]:
summary_sents

["Derek asks Judy if she can feed his animals on Friday and Saturday as she isn''t leaving for the weekend.",
 'She agrees and Derek plans to give her his keys on Thursday.',
 'He also promises to provide her with more details later.']